# Bibliotheken


In [7]:
import pandas as pd # Datenmanipulationsbibliothek
import numpy as np # Numerische Berechnungen
import matplotlib.pyplot as plt # Visualisierungen
from sklearn.preprocessing import MinMaxScaler # Zum Skalieren von Daten
import os # Für Betriebssystem-Interaktionen
import traceback # Für detaillierte Fehlermeldungen

# --- NEU: Hilfsfunktion zum Speichern einzelner Plots ---
def save_single_plot(fig, ax, output_folder, base_filename_part, plot_name_suffix):
    """
    Speichert eine Matplotlib-Figure in einer Datei und schließt sie.

    Args:
        fig (matplotlib.figure.Figure): Das Figure-Objekt.
        ax (matplotlib.axes.Axes): Das Axes-Objekt (für konsistente Einstellungen).
        output_folder (str): Der Zielordner.
        base_filename_part (str): Der Basis-Dateiname (z.B. plot_AAPL_stocks).
        plot_name_suffix (str): Der Suffix für die Kennzahl (z.B. BollingerBands).
    """
    ax.grid(True, alpha=0.3) # Konsistentes Gitter hinzufügen
    fig.tight_layout() # Layout anpassen
    plot_filename = os.path.join(output_folder, f"{base_filename_part}_{plot_name_suffix}.png")
    try:
        fig.savefig(plot_filename)
        print(f"Info: Plot gespeichert als {plot_filename}")
    except Exception as e:
        print(f"Fehler beim Speichern des Plots {plot_filename}: {e}")
    plt.close(fig) # Figure schließen, um Speicher freizugeben


def prepare_data(filepath, ticker, asset_type, output_folder): # output_folder als Argument hinzugefügt
    """
    Liest Finanzdaten aus einer CSV-Datei, bereinigt sie, führt Feature Engineering
    (SMAs, Bollinger Bänder, RSI, MACD) durch, visualisiert die Indikatoren inklusive Volumen
    in separaten PNG-Dateien und skaliert ausgewählte Features für potenzielle ML-Modelle.

    Args:
        filepath (str): Der Pfad zur Eingabe-CSV-Datei.
        ticker (str): Das Tickersymbol des Assets.
        asset_type (str): Der Typ des Assets (z.B. "Stocks", "ETFs").
        output_folder (str): Der Ordner zum Speichern der Plots und Daten.

    Returns:
        pandas.DataFrame or None: Ein DataFrame mit den vorbereiteten Daten und
                                  berechneten Features, oder None bei einem Fehler.
    """
    try:
        # --- Daten einlesen und Basisbereinigung ---
        # (Dieser Teil bleibt unverändert wie in der vorherigen Version)
        if not os.path.exists(filepath):
             print(f"Fehler: Datei '{filepath}' nicht gefunden.")
             return None
        stock_data = pd.read_csv(filepath, index_col='Date', parse_dates=True, thousands=',')
        required_cols = ['Volume', 'Open', 'Close', 'High', 'Low']
        for col in required_cols:
            if col not in stock_data.columns:
                print(f"FEHLER: Spalte '{col}' fehlt in Datei {filepath}. Verarbeitung abgebrochen.")
                return None
        adj_close_exists = 'Adj Close' in stock_data.columns
        if adj_close_exists: required_cols.append('Adj Close')

        # --- Datentyp-Konvertierung und Bereinigung ---
        # (Dieser Teil bleibt unverändert wie in der vorherigen Version)
        for col in required_cols:
            if pd.api.types.is_numeric_dtype(stock_data[col]): continue
            if stock_data[col].dtype == object:
                try:
                    stock_data[col] = pd.to_numeric(stock_data[col])
                    print(f"Info: Spalte '{col}' erfolgreich in numerisch konvertiert (direkt).")
                    continue
                except ValueError:
                    print(f"Info: Direkte Konvertierung von '{col}' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...")
                    temp_col = stock_data[col].astype(str).str.replace('M', '*10**6', regex=False).str.replace('B', '*10**9', regex=False)
                    def safe_eval(x):
                        try:
                            cleaned_x = ''.join(c for c in str(x) if c.isdigit() or c in ['.', 'e', 'E', '+', '-', '*'])
                            if not cleaned_x: return np.nan
                            return pd.eval(cleaned_x)
                        except Exception: return np.nan
                    evaluated_col = temp_col.apply(safe_eval)
                    if pd.api.types.is_numeric_dtype(evaluated_col):
                         stock_data[col] = evaluated_col
                         print(f"Info: Spalte '{col}' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.")
                    else:
                         stock_data[col] = pd.to_numeric(stock_data[col], errors='coerce')
                         print(f"Warnung: Konnte Spalte '{col}' nach eval nicht zuverlässig in numerisch konvertieren. Nicht-numerische Werte wurden zu NaN.")
            if not pd.api.types.is_numeric_dtype(stock_data[col]):
                 print(f"FEHLER: Spalte '{col}' konnte nicht in einen numerischen Typ konvertiert werden (Typ: {stock_data[col].dtype}). Verarbeitung abgebrochen.")
                 return None
        numeric_cols = stock_data.select_dtypes(include=np.number).columns
        stock_data[numeric_cols] = stock_data[numeric_cols].fillna(stock_data[numeric_cols].mean())
        stock_data.dropna(axis=1, how='all', inplace=True)

        # --- Feature Engineering ---
        # (Dieser Teil bleibt unverändert wie in der vorherigen Version)
        if "Close" not in stock_data.columns or not pd.api.types.is_numeric_dtype(stock_data["Close"]):
             print("FEHLER: Spalte 'Close' fehlt oder ist nicht numerisch nach der Bereinigung. Feature Engineering wird übersprungen.")
             return None
        stock_data["SMA_50"] = stock_data["Close"].rolling(window=50).mean()
        stock_data["SMA_200"] = stock_data["Close"].rolling(window=200).mean()
        stock_data['SMA_20'] = stock_data['Close'].rolling(window=20).mean()
        stock_data['StdDev_20'] = stock_data['Close'].rolling(window=20).std()
        stock_data['Upper'] = stock_data['SMA_20'] + 2 * stock_data['StdDev_20']
        stock_data['Lower'] = stock_data['SMA_20'] - 2 * stock_data['StdDev_20']
        delta = stock_data['Close'].diff()
        up = delta.clip(lower=0)
        down = -1 * delta.clip(upper=0)
        ema_up = up.ewm(com=13, adjust=False).mean()
        ema_down = down.ewm(com=13, adjust=False).mean()
        rs = ema_up / ema_down
        stock_data['RSI'] = 100 - (100 / (1 + rs))
        stock_data['RSI'] = stock_data['RSI'].replace([np.inf, -np.inf], 100)
        stock_data['EMA_12'] = stock_data['Close'].ewm(span=12, adjust=False).mean()
        stock_data['EMA_26'] = stock_data['Close'].ewm(span=26, adjust=False).mean()
        stock_data['MACD'] = stock_data['EMA_12'] - stock_data['EMA_26']
        stock_data['Signal'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()
        stock_data['MACD_Histogram'] = stock_data['MACD'] - stock_data['Signal']

        # --- NaN-Handling NACH Feature Engineering ---
        # (Dieser Teil bleibt unverändert wie in der vorherigen Version)
        initial_rows = len(stock_data)
        stock_data.dropna(inplace=True)
        print(f"Info {ticker}: {initial_rows - len(stock_data)} Zeilen am Anfang wegen Rolling-NaNs entfernt.")
        if stock_data.empty:
             print(f"FEHLER: Nach dem Entfernen von NaNs sind keine Daten mehr für {ticker} übrig.")
             return None

        # --- Visualisierung (separate Plots) ---
        print(f"Info: Erstelle separate Plots für {ticker}...")
        # Basis-Dateiname für die Plots dieses Tickers
        safe_ticker_name = ticker.replace('^', '')
        base_filename_part = f"plot_{safe_ticker_name}_{asset_type.lower()}"
        plot_figsize = (12, 6) # Standardgröße für einzelne Plots

        # Plot 1: Bollinger Bänder
        if all(col in stock_data.columns for col in ["Close", "SMA_20", "Upper", "Lower"]):
            fig_bb, ax_bb = plt.subplots(figsize=plot_figsize)
            ax_bb.plot(stock_data.index, stock_data["Close"], label="Schlusskurs", linewidth=1.5)
            ax_bb.plot(stock_data.index, stock_data["Upper"], label="Oberes Bollinger Band", color='red', linestyle='--', linewidth=1)
            ax_bb.plot(stock_data.index, stock_data["Lower"], label="Unteres Bollinger Band", color='green', linestyle='--', linewidth=1)
            ax_bb.plot(stock_data.index, stock_data["SMA_20"], label="SMA 20", color='orange', linestyle=':', linewidth=1)
            ax_bb.fill_between(stock_data.index, stock_data['Lower'], stock_data['Upper'], color='grey', alpha=0.1)
            ax_bb.set_title(f"Bollinger Bänder von {ticker}")
            ax_bb.set_ylabel("Preis")
            ax_bb.set_xlabel("Datum")
            ax_bb.legend()
            save_single_plot(fig_bb, ax_bb, output_folder, base_filename_part, "BollingerBands")
        else:
            print(f"Warnung {ticker}: Benötigte Spalten für Bollinger Band Plot fehlen.")

        # Plot 2: RSI
        if "RSI" in stock_data.columns:
            fig_rsi, ax_rsi = plt.subplots(figsize=plot_figsize)
            ax_rsi.plot(stock_data.index, stock_data["RSI"], label="RSI", color='purple', linewidth=1.5)
            ax_rsi.axhline(70, color='red', linestyle='--', label='Überkauft (70)', linewidth=1)
            ax_rsi.axhline(30, color='green', linestyle='--', label='Überverkauft (30)', linewidth=1)
            ax_rsi.set_title(f"Relative Strength Index (RSI) von {ticker}")
            ax_rsi.set_ylabel("RSI")
            ax_rsi.set_xlabel("Datum")
            ax_rsi.legend()
            ax_rsi.set_ylim(0, 100)
            save_single_plot(fig_rsi, ax_rsi, output_folder, base_filename_part, "RSI")
        else:
            print(f"Warnung {ticker}: Spalte 'RSI' fehlt für Plot.")

        # Plot 3: MACD
        if all(col in stock_data.columns for col in ["MACD", "Signal", "MACD_Histogram"]):
            fig_macd, ax_macd = plt.subplots(figsize=plot_figsize)
            macd_hist_colors = ['green' if val >= 0 else 'red' for val in stock_data['MACD_Histogram']]
            ax_macd.plot(stock_data.index, stock_data["MACD"], label="MACD", color='blue', linewidth=1.5)
            ax_macd.plot(stock_data.index, stock_data["Signal"], label="Signallinie", color='red', linestyle='--', linewidth=1.5)
            ax_macd.bar(stock_data.index, stock_data['MACD_Histogram'], label='MACD Histogramm', color=macd_hist_colors, alpha=0.6)
            ax_macd.axhline(0, color='grey', linestyle='-', linewidth=0.5)
            ax_macd.set_title(f"MACD von {ticker}")
            ax_macd.set_ylabel("MACD")
            ax_macd.set_xlabel("Datum")
            ax_macd.legend()
            save_single_plot(fig_macd, ax_macd, output_folder, base_filename_part, "MACD")
        else:
            print(f"Warnung {ticker}: Spalten für MACD Plot fehlen.")

        # Plot 4: Kurs und SMAs
        if "Close" in stock_data.columns:
            fig_sma, ax_sma = plt.subplots(figsize=plot_figsize)
            ax_sma.plot(stock_data.index, stock_data["Close"], label="Schlusskurs", linewidth=1.5)
            if "SMA_50" in stock_data.columns:
                ax_sma.plot(stock_data.index, stock_data["SMA_50"], label="SMA 50", color='orange', linestyle='--', linewidth=1)
            if "SMA_200" in stock_data.columns:
                ax_sma.plot(stock_data.index, stock_data["SMA_200"], label="SMA 200", color='magenta', linestyle=':', linewidth=1)
            ax_sma.set_title(f"Aktienkurs und Gleitende Durchschnitte von {ticker}")
            ax_sma.set_ylabel("Preis")
            ax_sma.set_xlabel("Datum")
            ax_sma.legend()
            save_single_plot(fig_sma, ax_sma, output_folder, base_filename_part, "Price_SMA")
        else:
             print(f"Warnung {ticker}: Spalte 'Close' fehlt für SMA Plot.")

        # Plot 5: Volumen Balkendiagramm
        if 'Volume' in stock_data.columns:
            fig_vol, ax_vol = plt.subplots(figsize=plot_figsize)
            if 'Open' in stock_data.columns and 'Close' in stock_data.columns:
                 volume_colors = ['green' if stock_data['Close'][i] >= stock_data['Open'][i] else 'red'
                                  for i in range(len(stock_data))]
            else:
                 volume_colors = 'blue'
            ax_vol.bar(stock_data.index, stock_data['Volume'], color=volume_colors, alpha=0.7, label='Volumen')
            ax_vol.set_title(f"Handelsvolumen von {ticker}")
            ax_vol.set_ylabel("Volumen")
            ax_vol.set_xlabel("Datum")
            # ax_vol.legend() # Legende bei Volumen oft nicht nötig
            save_single_plot(fig_vol, ax_vol, output_folder, base_filename_part, "Volume")
        else:
             print(f"Warnung {ticker}: Spalte 'Volume' fehlt für Plot.")


        # --- Datenvorbereitung für das Modell (Skalierung) ---
        # (Dieser Teil bleibt unverändert wie in der vorherigen Version)
        features_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume']
        if adj_close_exists and 'Adj Close' in stock_data.columns: features_to_scale.append('Adj Close')
        technical_indicators = ['SMA_50', 'SMA_200', 'SMA_20', 'StdDev_20', 'Upper', 'Lower', 'EMA_12', 'EMA_26', 'MACD', 'Signal', 'MACD_Histogram']
        for ind in technical_indicators:
            if ind in stock_data.columns: features_to_scale.append(ind)
        features_to_scale = [col for col in features_to_scale if col in stock_data.columns]
        if not features_to_scale:
             print(f"Warnung {ticker}: Keine Features zum Skalieren gefunden.")
        else:
             print(f"Info {ticker}: Skaliere Features: {features_to_scale}")
             scaler = MinMaxScaler()
             stock_data_scaled = stock_data.copy()
             stock_data_scaled[features_to_scale] = scaler.fit_transform(stock_data[features_to_scale])

        # --- Daten speichern (unskalierte Version mit Indikatoren) ---
        prepared_filename = f"prepared_{safe_ticker_name}_{asset_type.lower()}_data.csv" # Verwende sicheren Namen
        output_path = os.path.join(output_folder, prepared_filename)
        try:
             stock_data.to_csv(output_path)
             print(f"Info: Vorbereitete Daten gespeichert in {output_path}")
        except Exception as e:
             print(f"Fehler beim Speichern der CSV {output_path}: {e}")

    # --- Fehlerbehandlung ---
    # (Dieser Teil bleibt unverändert wie in der vorherigen Version)
    except FileNotFoundError:
        print(f"FEHLER: Datei '{filepath}' nicht gefunden.")
        return None
    except KeyError as e:
        print(f"FEHLER ({ticker}): Spaltenfehler - {e}. Möglicherweise fehlt eine benötigte Spalte nach der Bereinigung.")
        return None
    except Exception as e:
        print(f"FEHLER ({ticker}): Ein unerwarteter Fehler ist aufgetreten: {e}")
        print(traceback.format_exc())
        return None

    return stock_data


# --- Hauptteil des Skripts ---
# (Fast unverändert, nur output_folder wird an prepare_data übergeben)

output_folder = "financial_data"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Info: Ordner '{output_folder}' wurde erstellt.")

tickers = {
    "Stocks": ["AAPL", "MSFT", "GOOG"],
    "ETFs": ["ACWI", "SPY"],
    "Indices": ["^GSPC", "^DJI", "^IXIC"],
}

all_prepared_data = {}

for asset_type, ticker_list in tickers.items():
    print(f"\n--- Verarbeitung Asset-Typ: {asset_type} ---")
    for ticker in ticker_list:
        print(f"\nVerarbeite Ticker: {ticker}...")
        safe_ticker_name = ticker.replace('^', '')
        filename = f"{safe_ticker_name}_{asset_type.lower()}_data.csv"
        filepath = os.path.join(output_folder, filename)

        # *** WICHTIG: output_folder hier übergeben ***
        prepared_df = prepare_data(filepath, ticker, asset_type, output_folder)

        if prepared_df is None:
            print(f"FEHLER beim Vorbereiten der Daten für {ticker}. Fortfahren mit dem nächsten Ticker.")
            continue
        else:
            all_prepared_data[f"{ticker}_{asset_type}"] = prepared_df
            print(f"Erfolgreich verarbeitet: {ticker}")

# --- Optional: Alle Daten in eine Excel-Datei schreiben ---
# (Dieser Teil bleibt unverändert wie in der vorherigen Version)
excel_filename = os.path.join(output_folder, "alle_vorbereiteten_daten.xlsx")
if all_prepared_data:
    try:
        print(f"\nSchreibe alle vorbereiteten Daten in Excel-Datei: {excel_filename}")
        with pd.ExcelWriter(excel_filename) as writer:
            for sheet_name, df in all_prepared_data.items():
                 safe_sheet_name = sheet_name.replace('^', '').replace(':', '').replace('/', '')[:31]
                 df.to_excel(writer, sheet_name=safe_sheet_name)
        print("Excel-Datei erfolgreich geschrieben.")
    except Exception as e:
        print(f"FEHLER beim Schreiben der Excel-Datei '{excel_filename}': {e}")
else:
     print("\nKeine Daten erfolgreich verarbeitet, keine Excel-Datei erstellt.")

print("\n--- Skriptausführung beendet ---")


--- Verarbeitung Asset-Typ: Stocks ---

Verarbeite Ticker: AAPL...
Fehler: Datei 'financial_data\AAPL_stocks_data.csv' nicht gefunden.
FEHLER beim Vorbereiten der Daten für AAPL. Fortfahren mit dem nächsten Ticker.

Verarbeite Ticker: MSFT...
Fehler: Datei 'financial_data\MSFT_stocks_data.csv' nicht gefunden.
FEHLER beim Vorbereiten der Daten für MSFT. Fortfahren mit dem nächsten Ticker.

Verarbeite Ticker: GOOG...
Info: Direkte Konvertierung von 'Volume' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...
Info: Spalte 'Volume' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.
Info: Direkte Konvertierung von 'Open' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...
Info: Spalte 'Open' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.
Info: Direkte Konvertierung von 'Close' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...
Info: Spalte 'Close' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.
Info: Direkte Konvertieru

C:\Users\Aaron\AppData\Local\Temp\ipykernel_7548\2556698856.py:202: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  volume_colors = ['green' if stock_data['Close'][i] >= stock_data['Open'][i] else 'red'


Info: Plot gespeichert als financial_data\plot_GOOG_stocks_Volume.png
Info GOOG: Skaliere Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_50', 'SMA_200', 'SMA_20', 'StdDev_20', 'Upper', 'Lower', 'EMA_12', 'EMA_26', 'MACD', 'Signal', 'MACD_Histogram']
Info: Vorbereitete Daten gespeichert in financial_data\prepared_GOOG_stocks_data.csv
Erfolgreich verarbeitet: GOOG

--- Verarbeitung Asset-Typ: ETFs ---

Verarbeite Ticker: ACWI...
Info: Direkte Konvertierung von 'Volume' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...
Info: Spalte 'Volume' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.
Info: Direkte Konvertierung von 'Open' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...
Info: Spalte 'Open' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.
Info: Direkte Konvertierung von 'Close' fehlgeschlagen. Versuche 'M'/'B'-Ersetzung und Auswertung...
Info: Spalte 'Close' erfolgreich über 'M'/'B'-Ersetzung und Auswertung konvertiert.
Inf

C:\Users\Aaron\AppData\Local\Temp\ipykernel_7548\2556698856.py:202: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  volume_colors = ['green' if stock_data['Close'][i] >= stock_data['Open'][i] else 'red'


Info: Plot gespeichert als financial_data\plot_ACWI_etfs_Volume.png
Info ACWI: Skaliere Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_50', 'SMA_200', 'SMA_20', 'StdDev_20', 'Upper', 'Lower', 'EMA_12', 'EMA_26', 'MACD', 'Signal', 'MACD_Histogram']
Info: Vorbereitete Daten gespeichert in financial_data\prepared_ACWI_etfs_data.csv
Erfolgreich verarbeitet: ACWI

Verarbeite Ticker: SPY...
Fehler: Datei 'financial_data\SPY_etfs_data.csv' nicht gefunden.
FEHLER beim Vorbereiten der Daten für SPY. Fortfahren mit dem nächsten Ticker.

--- Verarbeitung Asset-Typ: Indices ---

Verarbeite Ticker: ^GSPC...
Fehler: Datei 'financial_data\GSPC_indices_data.csv' nicht gefunden.
FEHLER beim Vorbereiten der Daten für ^GSPC. Fortfahren mit dem nächsten Ticker.

Verarbeite Ticker: ^DJI...
Fehler: Datei 'financial_data\DJI_indices_data.csv' nicht gefunden.
FEHLER beim Vorbereiten der Daten für ^DJI. Fortfahren mit dem nächsten Ticker.

Verarbeite Ticker: ^IXIC...
Info: Direkte Konvertierung von 

C:\Users\Aaron\AppData\Local\Temp\ipykernel_7548\2556698856.py:202: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  volume_colors = ['green' if stock_data['Close'][i] >= stock_data['Open'][i] else 'red'


Info: Plot gespeichert als financial_data\plot_IXIC_indices_Volume.png
Info ^IXIC: Skaliere Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_50', 'SMA_200', 'SMA_20', 'StdDev_20', 'Upper', 'Lower', 'EMA_12', 'EMA_26', 'MACD', 'Signal', 'MACD_Histogram']
Info: Vorbereitete Daten gespeichert in financial_data\prepared_IXIC_indices_data.csv
Erfolgreich verarbeitet: ^IXIC

Schreibe alle vorbereiteten Daten in Excel-Datei: financial_data\alle_vorbereiteten_daten.xlsx
Excel-Datei erfolgreich geschrieben.

--- Skriptausführung beendet ---
